<h1 align="center"> ADA - Milestone 2</h1> 

**Students:**
- ABI FADEL Zad
- ADEYE Abiola
- BRUNO Etienne
- FERCHIOU Sami

In [ ]:
import os
import sqlite3

import numpy as np
# Import libraries
import pandas as pd
import seaborn as sns
from IPython.display import HTML, display

display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
pd.set_option("display.max_columns", None)
pd.options.display.max_rows = 20

%load_ext ipython_clipboard
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_json("data/video_metadata/yt_metadata_en.jsonl.gz", compression="gzip", chunksize=100000, lines=True)

In [ ]:
dfs = [pd.read_parquet(f"data/video_metadata/parquet/{i}") for i in os.listdir("data/video_metadata/parquet/")[:10]]  # only 10 first files to test

In [ ]:
df = pd.concat(dfs)
df.head(5)

<div class="alert alert-block alert-info">
    Creation and connection to a database that we will call youniverse. We will then fill it with the following data:  <br>
    <ul>
        <li> df_channels </li>
        <li> df_timeseries_en  </li>
        <li> num_comments  </li>
        <li> num_comments_authors  </li>
        <li> yt_metada_en  </li>
        <li> yt_metada_helper  </li>
    </ul>
 </div>

In [ ]:
# Connection to the main database
con = sqlite3.connect("databases/youniverse.sqlite")

In [ ]:
# Example of query
cur = con.cursor()
res = cur.execute("select count(*) from df_channels_en")
res.fetchone()

In [ ]:
# Outer loop containing subfolders of data
folders = os.listdir("data")
trash = ".DS_Store"
if trash in folders:
    folders.remove(trash)
folders.remove("ada_west_youniverse")
print(folders)

<div class="alert alert-block alert-info">
    <p>
    The following cell iterate over all data files. It considers only decompressed files, load them and save them into the aforementionned database called "youniverse.sqlite". </p>
    <p>
    We ignore the youtube_comments compressed file as we do not need it in our work and the yt_metadata_en file as we will use Dask to work efficiently with it.
    </p>
 </div>

In [ ]:
# Outer loop for subfolder
for folder in folders:
    # Remove .DS_Store file of macOS
    files = os.listdir(f"data/{folder}")
    if trash in files:
        files.remove(trash)

    print(files)
    # We only work with tsv or feather file since we have decompressed all gzip files
    # Inner loop for data files
    for file in files:
        name = file.split(".")[0]
        fmt = file.split(".")[1]
        file = "data/" + folder + "/" + file
        print(f"  -> reading {name} file in {fmt} format")
        if len(file.split(".")) != 2:  # Ignore compressed files (il still on computer)
            print(f"       >>> ignored")
            continue
        if name == "youtube_comments":  # Ignore for now the very large comment file
            print(f"       >>> ignored")
            continue
        if fmt == "tsv":
            df = pd.read_csv(file, sep="\t")
        elif fmt == "feather":
            df = pd.read_feather(file)
        elif fmt == "jsonl":  # Ignore the yt_metadata_en.jsonl file - we will use parquet files for it
            print(f"       >>> ignored")
            continue
        print(f"       >>> successful")

        # Save dataframe to SQL database
        df.to_sql(name, con, if_exists="replace")